In [ ]:
print('Listing current directory contents:')
!ls -F

In [ ]:
print('Searching for "core" directory within current directory and subdirectories:')
!find . -name 'core'

In [ ]:
print('Searching for "core" directory in parent directory and its subdirectories:')
!find .. -name 'core'

In [ ]:
import os

# Create the 'core' directory if it doesn't exist
if not os.path.exists('core'):
    os.makedirs('core')
    print("Created directory: core/")

# Create core/stock.py
stock_content = """
# core/stock.py

class Stock:
    def __init__(self, name, ticker, current_price):
        self.name = name
        self.ticker = ticker
        self.current_price = current_price

    def __repr__(self):
        return f"Stock(name='{self.name}', ticker='{self.ticker}', current_price={self.current_price})"
"""
with open('core/stock.py', 'w') as f:
    f.write(stock_content)
print("Created core/stock.py")

# Create core/cashflow_models.py
cashflow_models_content = """
# core/cashflow_models.py

def dcf_valuation(fcf_last, discount_rate, growth_rate, periods):
    # Simplified DCF model for illustration
    # Present Value of Future Cash Flows (P.V. of FCF) + Terminal Value (P.V. of TV)
    future_fcf = 0
    for i in range(1, periods + 1):
        future_fcf += fcf_last * ((1 + growth_rate)**i) / ((1 + discount_rate)**i)

    # Simplified Terminal Value (Gordon Growth Model)
    # Assuming growth rate continues beyond projected periods
    terminal_value = (fcf_last * (1 + growth_rate)**periods * (1 + growth_rate)) / (discount_rate - growth_rate)
    pv_terminal_value = terminal_value / ((1 + discount_rate)**periods)

    return future_fcf + pv_terminal_value

def ddm_valuation(dividend_next, discount_rate, growth_rate):
    # Gordon Growth Model for DDM
    if discount_rate <= growth_rate:
        return float('inf') # Or handle as an error/unsuitable model
    return dividend_next / (discount_rate - growth_rate)
"""
with open('core/cashflow_models.py', 'w') as f:
    f.write(cashflow_models_content)
print("Created core/cashflow_models.py")

# Create core/multiples.py
multiples_content = """
# core/multiples.py

def pe_valuation(eps, pe_ratio):
    return eps * pe_ratio

def pb_valuation(book_value, pb_ratio):
    return book_value * pb_ratio
"""
with open('core/multiples.py', 'w') as f:
    f.write(multiples_content)
print("Created core/multiples.py")

# Verify creation
!ls -F core/


In [ ]:

# Stock Valuation in Normal Markets

# 1️ Fix module import path
import sys
import os

# Function to find the project root containing the 'core' module
def find_project_root():
    current_dir = os.getcwd()
    # Check current directory
    if os.path.isdir(os.path.join(current_dir, 'core')):
        return current_dir
    # Check parent directory
    parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
    if os.path.isdir(os.path.join(parent_dir, 'core')):
        return parent_dir
    # Try common subdirectories of current_dir (e.g., if project is cloned into a subfolder)
    for sub_dir_name in os.listdir(current_dir):
        potential_root = os.path.join(current_dir, sub_dir_name)
        if os.path.isdir(potential_root) and os.path.isdir(os.path.join(potential_root, 'core')):
            return potential_root
    return None # Core module not found

# Add repo root to Python path (so 'core' can be found)
repo_root = find_project_root()
if repo_root is not None and repo_root not in sys.path:
    sys.path.append(repo_root)
elif repo_root is None:
    # Fallback: if core is not found in common places, try adding the current directory anyway.
    # This might catch cases where 'core' is handled differently or if the user intends current dir.
    if os.getcwd() not in sys.path:
        sys.path.append(os.getcwd())

# For debugging purposes, print the sys.path and identified repo_root
print(f"sys.path: {sys.path}")
print(f"repo_root identified: {repo_root}")


# 2️ Imports

import matplotlib.pyplot as plt
from core.stock import Stock
from core.cashflow_models import dcf_valuation, ddm_valuation
from core.multiples import pe_valuation, pb_valuation

# 3️ Sample Stock

apple = Stock("Apple Inc.", "AAPL", current_price=175)
print(f"Analyzing {apple.name} ({apple.ticker}) with current price ${apple.current_price}")

# 4️ DCF Valuation

fcf_last = 100e9        # Last free cash flow in $
discount_rate = 0.08    # 8% cost of capital
growth_rate = 0.05      # 5% annual growth
periods = 5             # Project next 5 years

dcf_value = dcf_valuation(fcf_last, discount_rate, growth_rate, periods)
print(f"DCF Intrinsic Value: ${dcf_value/1e9:.2f} Billion")


# 5️ DDM Valuation

dividend_next = 1.0     # Expected next dividend $ per share
ddm_value = ddm_valuation(dividend_next, discount_rate, growth_rate)
print(f"DDM Intrinsic Value: ${ddm_value:.2f} per share")

# 6️ Relative Valuation

eps = 5.0               # Earnings per share
book_value = 20.0       # Book value per share
pe_ratio = 30           # P/E ratio from comparable companies
pb_ratio = 8            # P/B ratio from comparable companies

pe_value = pe_valuation(eps, pe_ratio)
pb_value = pb_valuation(book_value, pb_ratio)
print(f"P/E Valuation: ${pe_value:.2f} per share")
print(f"P/B Valuation: ${pb_value:.2f} per share")


# 7️ Compare Valuations

methods = ['DCF (Billion $)', 'DDM', 'P/E', 'P/B']
values = [dcf_value/1e9, ddm_value, pe_value, pb_value]

plt.figure(figsize=(8,5))
plt.bar(methods, values, color=['blue', 'green', 'orange', 'red'])
plt.title(f"{apple.name} Stock Valuation - Normal Market")
plt.ylabel("Value")
plt.show()